# Array operations

Array arithmetic require more care than scalar arithmetic, the main point of differentiation is whether you want array operation (where defined) or element-wise operation.
For example, if A and B are two matrices, when you multiply them, do you mean Linear Algebra Matrix Multiply, or do you mean scalar multiply each corresponding element.

## Matrix operators

Julia has the following array operators:

Expression | Name | Description
---------- | --------- | ----------------------------
+x | unary plus | identity
-x | unary minus | element-wise change sign
x + y | binary plus | element-wise addition
x - y | binary minus | element-wise subtraction
x * y | times | matrix multiply
x ^ y | power | matrix raises to the yth power
x / y | divide | matrix division
x \ y | inverse divide | equivalent to y / x

Addition and subtraction are element-wise; multiply, divide and power are Linear Algebra matrix operations.

In [2]:
a = [1. 3; 2 4]

2×2 Array{Float64,2}:
 1.0  3.0
 2.0  4.0

In [17]:
b = Float64[
    1 1
    0 1
]

2×2 Array{Float64,2}:
 1.0  1.0
 0.0  1.0

In [19]:
a + b                         # element-wise

2×2 Array{Float64,2}:
 2.0  4.0
 2.0  5.0

In [20]:
a - b                         # element-wise

2×2 Array{Float64,2}:
 0.0  2.0
 2.0  3.0

In [21]:
a * b                         # matrix multiply

2×2 Array{Float64,2}:
 1.0  4.0
 2.0  6.0

In [22]:
a^2                           # matrix power

2×2 Array{Float64,2}:
  7.0  15.0
 10.0  22.0

In [24]:
a ^ 0.5                       # square root

2×2 Array{Complex{Float64},2}:
 0.553689+0.464394im  1.21044-0.31864im
 0.806961-0.212426im  1.76413+0.145754im

In [33]:
b ^ -1                        # matrix inverse

2×2 Array{Float64,2}:
 1.0  -1.0
 0.0   1.0

In [34]:
a / b                         # division = multiply by inverse

2×2 Array{Float64,2}:
 1.0  2.0
 2.0  2.0

## Comparison operators

Matrix comparison operators all operate element-wise:

Expression | Name 
---------- | --------- 
== | all element equal
!= | any element not equal
≈  | all element approximately equal (\approx<tab>
≉  | not all element approximately equal (\napprox<tab>)

In [12]:
a == b

false

In [13]:
a == a

true

In [14]:
a ≈ a

true

## Dot syntax for vectorizing functions

Technical computing languages have a function that operates on a scalar and another vectorized function that would operate element-wise over an array, object-oriented function dispatch would pick the right version for user convenience.
This approach is often required not just for ease of use but for performance: loops are slow in the language, the vectorized version will call fast library code written in a low-level language.

Julia's loops are fast, vectorized versions of functions are not required for performance. You can loop over the array or use iterators over the array.
However, to retain ease of use, **any Julia function f can be applied elementwise to any array (or other collection) with the syntax f.(a)**.
And, in Julia, all operators (arithmetic, bitwise, comparison) are functions. (This dot notation first appeared in MATLAB.)

In [16]:
x = [5, 6, 7, 8]    # array

4-element Array{Int64,1}:
 5
 6
 7
 8

In [17]:
y = [1, 2, 3, 4]

4-element Array{Int64,1}:
 1
 2
 3
 4

In [18]:
x .+ y

4-element Array{Int64,1}:
  6
  8
 10
 12

In [19]:
x .- y

4-element Array{Int64,1}:
 4
 4
 4
 4

In [20]:
x ./ y              # not the same as x / y !

4-element Array{Float64,1}:
 5.0
 3.0
 2.3333333333333335
 2.0

In [21]:
x .^ y              # x ^ y not defined

4-element Array{Int64,1}:
    5
   36
  343
 4096

The expression `x .^ y` is parsed as `(^).(x, y)` and it will broadcast the operands so that it can combine arrays and scalars, arrays of the same size, and arrays of different shape:

In [22]:
x .+ 10             # x + 10 doesn't work

4-element Array{Int64,1}:
 15
 16
 17
 18

In [23]:
.-x                 # element-wise negative

4-element Array{Int64,1}:
 -5
 -6
 -7
 -8

In [24]:
x .<= 6

4-element BitArray{1}:
 1
 1
 0
 0

In [25]:
.√x                 # element-wise square root

4-element Array{Float64,1}:
 2.23606797749979
 2.449489742783178
 2.6457513110645907
 2.8284271247461903

In [26]:
x .^ 3

4-element Array{Int64,1}:
 125
 216
 343
 512

In [27]:
sin.(x)

4-element Array{Float64,1}:
 -0.9589242746631385
 -0.27941549819892586
  0.6569865987187891
  0.9893582466233818

In [28]:
sin.(cos.(x))                      # fused broadcast

4-element Array{Float64,1}:
  0.2798733507685274
  0.819289219220601
  0.6844887989926141
 -0.14498719803267052

## Fused broadcast

The dot notation is a form of **broadcast** operation over arrays of different sizes.
Another very important performance advantage is Julia's broadcast of a dot expression is **all broadcast operation is fused into a single loop**.
Other languages would separately loop over each operator and pass temporary array between them to build up the result sequentially.

## @. macro

If you have a long expression, having many dots makes it hard to read and understand.
Julia has a `@.` macro call which will replace all operators and function calls with the dot vectorized form, e.g.,

In [29]:
3 .* x .^2 .+ 5 .* x .+ exp.(cos.(x)) .- 2.5      # many dots, an eye sore

4-element Array{Float64,1}:
  98.82798424258861
 138.1121412483359
 181.62527722851183
 230.36458986084338

The `@.` macro makes the expression far easier to read:

In [30]:
@. 3x^2 + 5x + exp(cos(x)) - 2.5                 # natural to read

4-element Array{Float64,1}:
  98.82798424258861
 138.1121412483359
 181.62527722851183
 230.36458986084338

Because dot is also used in floating point literals, spaces must be used to disambiguate them. e.g.,
does `1.+x` mean `1. + x` or `1 .+ x`?

## Caveat

Julia has many functions with similar names that can be confusing at first brush. A good example is maximum:

- `maximum(f, itr)`  returns the largest result of f(itr)
- `maximum(itr)` returns the largest of itr
- `maximum(A::AbstractArray; dims)` returns the largest over the specified dims of A
- `max(x, y, ...)` returns the maximum of the arguments

Be clear on what you want and what did you ask for.

## Performance tips

- If you are updating the same vector over and over, pre-allocate the result vector once and update it in place with .= assignment

```
y = Array{}...
x = similar(y)
x .= <expression>
```

- Because dot broadcast is fused, rather than array arithmetic such as vector + vector, use vector .+ vector because it can then be fused with surrounding operations

- There's a [StaticArrays](https://juliaarrays.github.io/StaticArrays.jl/stable/) package for statically sized arrays that have fast implementations of common array and linear algebra operations. For arrays smaller than 100 elements, StaticArrays can be up to an order faster than Julia base arrays.

- Read this [more dots blog](https://julialang.org/blog/2017/01/moredots)